In [1]:
%load_ext autoreload
%autoreload 1
%aimport redditClient

In [1]:
import networkx as nx

from redditClient import redditClient

import matplotlib.pyplot as plt
import time
import random

In [3]:
nvidiaFileName = "nvidia.graphml"
amdFileName = "amd.graphml"
intelFileName = "intel.graphml"

In [4]:
client = redditClient()

In [95]:

subreddits_nvidia = client.subreddit("nvidia+graphicscard+gpu+buildapc")


In [97]:
# gpu_keywords_nvidia = {
#     "rtx 4060": ("nvidia", "low"),
#     "rtx 4070": ("nvidia", "mid"),
#     "rtx 4070 super": ("nvidia", "mid"),
#     "rtx 4070 ti super": ("nvidia", "mid"),
#     "rtx 4080": ("nvidia", "high"),
#     "rtx 4090": ("nvidia", "high")
# }

# search_phrases_nvidia = []
# for gpu in gpu_keywords_nvidia.keys():
#     search_phrases_nvidia += [f"{gpu} review", f"{gpu} launch"]

from itertools import product

gpu_keywords_nvidia = {
    "rtx 4060": ("nvidia", "low")
#     "rtx 4070": ("nvidia", "mid"),
#     "rtx 4070 super": ("nvidia", "mid"),
#     "rtx 4070 ti super": ("nvidia", "mid"),
#     "rtx 4080": ("nvidia", "high"),
#     "rtx 4090": ("nvidia", "high")
}

contexts = ["", "review", "launch", "performance", "benchmarks", "gaming", "issues", "worth it", "upgrade", "vs"]

search_phrases_nvidia = [f"{gpu} {ctx}".strip() for gpu, ctx in product(gpu_keywords_nvidia.keys(), contexts)]

In [98]:
search_phrases_nvidia

['rtx 4060',
 'rtx 4060 review',
 'rtx 4060 launch',
 'rtx 4060 performance',
 'rtx 4060 benchmarks',
 'rtx 4060 gaming',
 'rtx 4060 issues',
 'rtx 4060 worth it',
 'rtx 4060 upgrade',
 'rtx 4060 vs']

## Nvidia

In [100]:
replyGraph = nx.DiGraph()
dSubCommentId = {}
seen_ids = set()

for phrase in search_phrases_nvidia:
    print(f"🔍 Processing search: {phrase}")

    for submission in subreddit.search(query=phrase, sort="relevance", time_filter="all", limit=20):
        if submission.id in seen_ids or not submission.author:
            continue
        seen_ids.add(submission.id)

        author_name = submission.author.name
        replyGraph.add_node(author_name, subNum=1)
        dSubCommentId[submission.id] = author_name

        try:
            submission.comments.replace_more(limit=None)
            all_comments = submission.comments.list()

    
            sorted_comments = sorted(
                [c for c in all_comments if c.author],
                key=lambda x: x.score,
                reverse=True
            )[:200]

            # First pass: index authors
            for comment in sorted_comments:
                dSubCommentId[comment.id] = comment.author.name

            # Second pass: add edges
            for comment in sorted_comments:
                parent_id = comment.parent_id.replace("t1_", "").replace("t3_", "")
                if parent_id in dSubCommentId:
                    src = comment.author.name
                    tgt = dSubCommentId[parent_id]

                    if src not in replyGraph:
                        replyGraph.add_node(src, subNum=0)
                    if tgt not in replyGraph:
                        replyGraph.add_node(tgt, subNum=0)

                    if replyGraph.has_edge(src, tgt):
                        replyGraph[src][tgt]['replyNum'] += 1
                    else:
                        replyGraph.add_edge(src, tgt, replyNum=1)

        except Exception as e:
            print(f"Skipped submission {submission.id} due to error: {e}")

        time.sleep(3)  # Avoid rate limit

# --- Save graph ---
nx.write_graphml(replyGraph, "search_reply_graph_4.graphml")
print(f"Graph saved with {len(replyGraph.nodes)} nodes and {len(replyGraph.edges)} edges.")

🔍 Processing search: rtx 4060
🔍 Processing search: rtx 4060 review
🔍 Processing search: rtx 4060 launch
🔍 Processing search: rtx 4060 performance
🔍 Processing search: rtx 4060 benchmarks
🔍 Processing search: rtx 4060 gaming
🔍 Processing search: rtx 4060 issues
🔍 Processing search: rtx 4060 worth it
🔍 Processing search: rtx 4060 upgrade
🔍 Processing search: rtx 4060 vs
✅ Graph saved with 5521 nodes and 9587 edges.


In [101]:
# 2ND ONE 

from itertools import product

gpu_keywords_nvidia = {
#     "rtx 4060": ("nvidia", "low")
#     "rtx 4070": ("nvidia", "mid"),
#     "rtx 4070 super": ("nvidia", "mid"),
    "rtx 4070 ti super": ("nvidia", "mid")
#     "rtx 4080": ("nvidia", "high"),
#     "rtx 4090": ("nvidia", "high")
}

contexts = ["", "review", "launch", "performance", "benchmarks", "gaming", "issues", "worth it", "upgrade", "vs"]

search_phrases_nvidia_1 = [f"{gpu} {ctx}".strip() for gpu, ctx in product(gpu_keywords_nvidia.keys(), contexts)]

In [104]:
del replyGraph 
del dSubCommentId

In [105]:
replyGraph = nx.DiGraph()
dSubCommentId = {}
seen_ids = set()

for phrase in search_phrases_nvidia_1:
    print(f"🔍 Processing search: {phrase}")

    for submission in subreddit.search(query=phrase, sort="relevance", time_filter="all", limit=20):
        if submission.id in seen_ids or not submission.author:
            continue
        seen_ids.add(submission.id)

        author_name = submission.author.name
        replyGraph.add_node(author_name, subNum=1)
        dSubCommentId[submission.id] = author_name

        try:
            submission.comments.replace_more(limit=None)
            all_comments = submission.comments.list()

            # Sort comments by score, descending, and take top 300
            sorted_comments = sorted(
                [c for c in all_comments if c.author],
                key=lambda x: x.score,
                reverse=True
            )[:200]

            # First pass: index authors
            for comment in sorted_comments:
                dSubCommentId[comment.id] = comment.author.name

            # Second pass: add edges
            for comment in sorted_comments:
                parent_id = comment.parent_id.replace("t1_", "").replace("t3_", "")
                if parent_id in dSubCommentId:
                    src = comment.author.name
                    tgt = dSubCommentId[parent_id]

                    if src not in replyGraph:
                        replyGraph.add_node(src, subNum=0)
                    if tgt not in replyGraph:
                        replyGraph.add_node(tgt, subNum=0)

                    if replyGraph.has_edge(src, tgt):
                        replyGraph[src][tgt]['replyNum'] += 1
                    else:
                        replyGraph.add_edge(src, tgt, replyNum=1)

        except Exception as e:
            print(f"⚠️ Skipped submission {submission.id} due to error: {e}")

        time.sleep(3)  # Avoid rate limit

# --- Save graph ---
nx.write_graphml(replyGraph, "search_reply_graph_5.graphml")
print(f"✅ Graph saved with {len(replyGraph.nodes)} nodes and {len(replyGraph.edges)} edges.")

🔍 Processing search: rtx 4070 ti super
🔍 Processing search: rtx 4070 ti super review
🔍 Processing search: rtx 4070 ti super launch
🔍 Processing search: rtx 4070 ti super performance
🔍 Processing search: rtx 4070 ti super benchmarks
🔍 Processing search: rtx 4070 ti super gaming
🔍 Processing search: rtx 4070 ti super issues
🔍 Processing search: rtx 4070 ti super worth it
🔍 Processing search: rtx 4070 ti super upgrade
🔍 Processing search: rtx 4070 ti super vs
✅ Graph saved with 4517 nodes and 7570 edges.


In [106]:
# 3rd ONE 

from itertools import product

gpu_keywords_nvidia = {
#     "rtx 4060": ("nvidia", "low")
#     "rtx 4070": ("nvidia", "mid"),
#     "rtx 4070 super": ("nvidia", "mid"),
    "rtx 4070 ti super": ("nvidia", "mid")
#     "rtx 4080": ("nvidia", "high"),
#     "rtx 4090": ("nvidia", "high")
}

contexts = ["", "review", "launch", "performance", "benchmarks", "gaming", "issues", "worth it", "upgrade", "vs"]

search_phrases_nvidia_2 = [f"{gpu} {ctx}".strip() for gpu, ctx in product(gpu_keywords_nvidia.keys(), contexts)]

In [109]:
del replyGraph 
del dSubCommentId

In [110]:
replyGraph = nx.DiGraph()
dSubCommentId = {}
seen_ids = set()

for phrase in search_phrases_nvidia_2:
    print(f"🔍 Processing search: {phrase}")

    for submission in subreddit.search(query=phrase, sort="relevance", time_filter="all", limit=20):
        if submission.id in seen_ids or not submission.author:
            continue
        seen_ids.add(submission.id)

        author_name = submission.author.name
        replyGraph.add_node(author_name, subNum=1)
        dSubCommentId[submission.id] = author_name

        try:
            submission.comments.replace_more(limit=None)
            all_comments = submission.comments.list()

            # Sort comments by score, descending, and take top 300
            sorted_comments = sorted(
                [c for c in all_comments if c.author],
                key=lambda x: x.score,
                reverse=True
            )[:200]

            # First pass: index authors
            for comment in sorted_comments:
                dSubCommentId[comment.id] = comment.author.name

            # Second pass: add edges
            for comment in sorted_comments:
                parent_id = comment.parent_id.replace("t1_", "").replace("t3_", "")
                if parent_id in dSubCommentId:
                    src = comment.author.name
                    tgt = dSubCommentId[parent_id]

                    if src not in replyGraph:
                        replyGraph.add_node(src, subNum=0)
                    if tgt not in replyGraph:
                        replyGraph.add_node(tgt, subNum=0)

                    if replyGraph.has_edge(src, tgt):
                        replyGraph[src][tgt]['replyNum'] += 1
                    else:
                        replyGraph.add_edge(src, tgt, replyNum=1)

        except Exception as e:
            print(f"⚠️ Skipped submission {submission.id} due to error: {e}")

        time.sleep(3)  # Avoid rate limit

# --- Save graph ---
nx.write_graphml(replyGraph, "search_reply_graph_6.graphml")
print(f"✅ Graph saved with {len(replyGraph.nodes)} nodes and {len(replyGraph.edges)} edges.")

🔍 Processing search: rtx 4070 ti super
🔍 Processing search: rtx 4070 ti super review
🔍 Processing search: rtx 4070 ti super launch
🔍 Processing search: rtx 4070 ti super performance
🔍 Processing search: rtx 4070 ti super benchmarks
🔍 Processing search: rtx 4070 ti super gaming
🔍 Processing search: rtx 4070 ti super issues
🔍 Processing search: rtx 4070 ti super worth it
🔍 Processing search: rtx 4070 ti super upgrade
🔍 Processing search: rtx 4070 ti super vs
✅ Graph saved with 4513 nodes and 7561 edges.


# AMD


In [111]:
subreddits_amd = client.subreddit("radeon+graphicscard+gpu+buildapc")

In [146]:
gpu_keywords_amd = {
    "rx 6500 xt": ("amd", "low")
#     "rx 7600": ("amd", "mid"),
#     "rx 7900 xt": ("amd", "high")
}

In [147]:
# 2ND ONE 

from itertools import product


contexts = ["", "review", "launch", "performance", "benchmarks", "gaming", "issues", "worth it", "upgrade", "vs"]

search_phrases_amd_1 = [f"{gpu} {ctx}".strip() for gpu, ctx in product(gpu_keywords_amd.keys(), contexts)]

In [148]:
search_phrases_amd_1

['rx 6500 xt',
 'rx 6500 xt review',
 'rx 6500 xt launch',
 'rx 6500 xt performance',
 'rx 6500 xt benchmarks',
 'rx 6500 xt gaming',
 'rx 6500 xt issues',
 'rx 6500 xt worth it',
 'rx 6500 xt upgrade',
 'rx 6500 xt vs']

In [137]:
del replyGraph
del dSubCommentId 
del seen_ids

NameError: name 'replyGraph' is not defined

In [149]:
replyGraph = nx.DiGraph()
dSubCommentId = {}
seen_ids = set()

for phrase in search_phrases_amd_1:
    print(f"🔍 Processing search: {phrase}")

    for submission in subreddits_amd.search(query=phrase, sort="relevance", time_filter="all", limit=20):
        if submission.id in seen_ids or not submission.author:
            continue
        seen_ids.add(submission.id)

        author_name = submission.author.name
        replyGraph.add_node(author_name, subNum=1)
        dSubCommentId[submission.id] = author_name

        try:
            submission.comments.replace_more(limit=None)
            all_comments = submission.comments.list()

            # Sort comments by score, descending, and take top 300
            sorted_comments = sorted(
                [c for c in all_comments if c.author],
                key=lambda x: x.score,
                reverse=True
            )[:200]

            # First pass: index authors
            for comment in sorted_comments:
                dSubCommentId[comment.id] = comment.author.name

            # Second pass: add edges
            for comment in sorted_comments:
                parent_id = comment.parent_id.replace("t1_", "").replace("t3_", "")
                if parent_id in dSubCommentId:
                    src = comment.author.name
                    tgt = dSubCommentId[parent_id]

                    if src not in replyGraph:
                        replyGraph.add_node(src, subNum=0)
                    if tgt not in replyGraph:
                        replyGraph.add_node(tgt, subNum=0)

                    if replyGraph.has_edge(src, tgt):
                        replyGraph[src][tgt]['replyNum'] += 1
                    else:
                        replyGraph.add_edge(src, tgt, replyNum=1)

        except Exception as e:
            print(f"⚠️ Skipped submission {submission.id} due to error: {e}")

        time.sleep(3)  # Avoid rate limit

# --- Save graph ---
nx.write_graphml(replyGraph, "search_reply_graph_amd_1.graphml")
print(f"✅ Graph saved with {len(replyGraph.nodes)} nodes and {len(replyGraph.edges)} edges.")

🔍 Processing search: rx 6500 xt
🔍 Processing search: rx 6500 xt review
🔍 Processing search: rx 6500 xt launch
🔍 Processing search: rx 6500 xt performance
🔍 Processing search: rx 6500 xt benchmarks
🔍 Processing search: rx 6500 xt gaming
🔍 Processing search: rx 6500 xt issues
🔍 Processing search: rx 6500 xt worth it
🔍 Processing search: rx 6500 xt upgrade
🔍 Processing search: rx 6500 xt vs
✅ Graph saved with 1638 nodes and 2339 edges.


In [150]:
gpu_keywords_amd = {
#     "rx 6500 xt": ("amd", "low")
    "rx 7600": ("amd", "mid")
#     "rx 7900 xt": ("amd", "high")
}


from itertools import product


contexts = ["", "review", "launch", "performance", "benchmarks", "gaming", "issues", "worth it", "upgrade", "vs"]

search_phrases_amd_2 = [f"{gpu} {ctx}".strip() for gpu, ctx in product(gpu_keywords_amd.keys(), contexts)]

In [151]:
del replyGraph
del dSubCommentId 
del seen_ids

In [152]:
# amd 2 
replyGraph = nx.DiGraph()
dSubCommentId = {}
seen_ids = set()

for phrase in search_phrases_amd_2:
    print(f"🔍 Processing search: {phrase}")

    for submission in subreddits_amd.search(query=phrase, sort="relevance", time_filter="all", limit=20):
        if submission.id in seen_ids or not submission.author:
            continue
        seen_ids.add(submission.id)

        author_name = submission.author.name
        replyGraph.add_node(author_name, subNum=1)
        dSubCommentId[submission.id] = author_name

        try:
            submission.comments.replace_more(limit=None)
            all_comments = submission.comments.list()

            # Sort comments by score, descending, and take top 300
            sorted_comments = sorted(
                [c for c in all_comments if c.author],
                key=lambda x: x.score,
                reverse=True
            )[:200]

            # First pass: index authors
            for comment in sorted_comments:
                dSubCommentId[comment.id] = comment.author.name

            # Second pass: add edges
            for comment in sorted_comments:
                parent_id = comment.parent_id.replace("t1_", "").replace("t3_", "")
                if parent_id in dSubCommentId:
                    src = comment.author.name
                    tgt = dSubCommentId[parent_id]

                    if src not in replyGraph:
                        replyGraph.add_node(src, subNum=0)
                    if tgt not in replyGraph:
                        replyGraph.add_node(tgt, subNum=0)

                    if replyGraph.has_edge(src, tgt):
                        replyGraph[src][tgt]['replyNum'] += 1
                    else:
                        replyGraph.add_edge(src, tgt, replyNum=1)

        except Exception as e:
            print(f"⚠️ Skipped submission {submission.id} due to error: {e}")

        time.sleep(3)  # Avoid rate limit

# --- Save graph ---
nx.write_graphml(replyGraph, "search_reply_graph_amd_2.graphml")
print(f"✅ Graph saved with {len(replyGraph.nodes)} nodes and {len(replyGraph.edges)} edges.")

🔍 Processing search: rx 7600
🔍 Processing search: rx 7600 review
🔍 Processing search: rx 7600 launch
🔍 Processing search: rx 7600 performance
🔍 Processing search: rx 7600 benchmarks
🔍 Processing search: rx 7600 gaming
🔍 Processing search: rx 7600 issues
🔍 Processing search: rx 7600 worth it
🔍 Processing search: rx 7600 upgrade
🔍 Processing search: rx 7600 vs
✅ Graph saved with 1897 nodes and 2694 edges.


In [153]:
del replyGraph
del dSubCommentId 

In [154]:
gpu_keywords_amd = {
#     "rx 6500 xt": ("amd", "low")
#     "rx 7600": ("amd", "mid")
    "rx 7900 xt": ("amd", "high")
}


from itertools import product


contexts = ["", "review", "launch", "performance", "benchmarks", "gaming", "issues", "worth it", "upgrade", "vs"]

search_phrases_amd_3 = [f"{gpu} {ctx}".strip() for gpu, ctx in product(gpu_keywords_amd.keys(), contexts)]

In [155]:
# amd 2 
replyGraph = nx.DiGraph()
dSubCommentId = {}
seen_ids = set()

for phrase in search_phrases_amd_3:
    print(f"🔍 Processing search: {phrase}")

    for submission in subreddits_amd.search(query=phrase, sort="relevance", time_filter="all", limit=20):
        if submission.id in seen_ids or not submission.author:
            continue
        seen_ids.add(submission.id)

        author_name = submission.author.name
        replyGraph.add_node(author_name, subNum=1)
        dSubCommentId[submission.id] = author_name

        try:
            submission.comments.replace_more(limit=None)
            all_comments = submission.comments.list()

            # Sort comments by score, descending, and take top 300
            sorted_comments = sorted(
                [c for c in all_comments if c.author],
                key=lambda x: x.score,
                reverse=True
            )[:200]

            # First pass: index authors
            for comment in sorted_comments:
                dSubCommentId[comment.id] = comment.author.name

            # Second pass: add edges
            for comment in sorted_comments:
                parent_id = comment.parent_id.replace("t1_", "").replace("t3_", "")
                if parent_id in dSubCommentId:
                    src = comment.author.name
                    tgt = dSubCommentId[parent_id]

                    if src not in replyGraph:
                        replyGraph.add_node(src, subNum=0)
                    if tgt not in replyGraph:
                        replyGraph.add_node(tgt, subNum=0)

                    if replyGraph.has_edge(src, tgt):
                        replyGraph[src][tgt]['replyNum'] += 1
                    else:
                        replyGraph.add_edge(src, tgt, replyNum=1)

        except Exception as e:
            print(f"⚠️ Skipped submission {submission.id} due to error: {e}")

        time.sleep(3)  # Avoid rate limit

# --- Save graph ---
nx.write_graphml(replyGraph, "search_reply_graph_amd_3.graphml")
print(f"✅ Graph saved with {len(replyGraph.nodes)} nodes and {len(replyGraph.edges)} edges.")

🔍 Processing search: rx 7900 xt
🔍 Processing search: rx 7900 xt review
🔍 Processing search: rx 7900 xt launch
🔍 Processing search: rx 7900 xt performance
🔍 Processing search: rx 7900 xt benchmarks
🔍 Processing search: rx 7900 xt gaming
🔍 Processing search: rx 7900 xt issues
🔍 Processing search: rx 7900 xt worth it
🔍 Processing search: rx 7900 xt upgrade
🔍 Processing search: rx 7900 xt vs
✅ Graph saved with 4218 nodes and 7037 edges.


## intel

In [159]:
subreddits_intel = client.subreddit("intelarc+graphicscard+gpu+buildapc")

In [162]:
gpu_keywords_intel = {
    "arc a380": ("intel", "low")
#     "arc a750": ("intel", "mid"),
#     "arc b580": ("intel", "high")
}



from itertools import product


contexts = ["", "review", "launch", "performance", "benchmarks", "gaming", "issues", "worth it", "upgrade", "vs"]

search_phrases_intel_1 = [f"{gpu} {ctx}".strip() for gpu, ctx in product(gpu_keywords_intel.keys(), contexts)]


In [163]:
del replyGraph
del dSubCommentId 

In [165]:
# intel 1 
replyGraph = nx.DiGraph()
dSubCommentId = {}
seen_ids = set()

for phrase in search_phrases_intel_1:
    print(f"🔍 Processing search: {phrase}")

    for submission in subreddits_intel.search(query=phrase, sort="relevance", time_filter="all", limit=20):
        if submission.id in seen_ids or not submission.author:
            continue
        seen_ids.add(submission.id)

        author_name = submission.author.name
        replyGraph.add_node(author_name, subNum=1)
        dSubCommentId[submission.id] = author_name

        try:
            submission.comments.replace_more(limit=None)
            all_comments = submission.comments.list()

            # Sort comments by score, descending, and take top 300
            sorted_comments = sorted(
                [c for c in all_comments if c.author],
                key=lambda x: x.score,
                reverse=True
            )[:200]

            # First pass: index authors
            for comment in sorted_comments:
                dSubCommentId[comment.id] = comment.author.name

            # Second pass: add edges
            for comment in sorted_comments:
                parent_id = comment.parent_id.replace("t1_", "").replace("t3_", "")
                if parent_id in dSubCommentId:
                    src = comment.author.name
                    tgt = dSubCommentId[parent_id]

                    if src not in replyGraph:
                        replyGraph.add_node(src, subNum=0)
                    if tgt not in replyGraph:
                        replyGraph.add_node(tgt, subNum=0)

                    if replyGraph.has_edge(src, tgt):
                        replyGraph[src][tgt]['replyNum'] += 1
                    else:
                        replyGraph.add_edge(src, tgt, replyNum=1)

        except Exception as e:
            print(f"⚠️ Skipped submission {submission.id} due to error: {e}")

        time.sleep(3)  # Avoid rate limit

# --- Save graph ---
nx.write_graphml(replyGraph, "search_reply_graph_intel_1.graphml")
print(f"✅ Graph saved with {len(replyGraph.nodes)} nodes and {len(replyGraph.edges)} edges.")

🔍 Processing search: arc a380
🔍 Processing search: arc a380 review
🔍 Processing search: arc a380 launch
🔍 Processing search: arc a380 performance
🔍 Processing search: arc a380 benchmarks
🔍 Processing search: arc a380 gaming
🔍 Processing search: arc a380 issues
🔍 Processing search: arc a380 worth it
🔍 Processing search: arc a380 upgrade
🔍 Processing search: arc a380 vs
✅ Graph saved with 821 nodes and 1279 edges.


## intel 2

In [166]:
gpu_keywords_intel = {
#     "arc a380": ("intel", "low")
    "arc a750": ("intel", "mid")
#     "arc b580": ("intel", "high")
}



from itertools import product


contexts = ["", "review", "launch", "performance", "benchmarks", "gaming", "issues", "worth it", "upgrade", "vs"]

search_phrases_intel_2 = [f"{gpu} {ctx}".strip() for gpu, ctx in product(gpu_keywords_intel.keys(), contexts)]


In [167]:
del replyGraph
del dSubCommentId 

In [168]:
# intel 1 
replyGraph = nx.DiGraph()
dSubCommentId = {}
seen_ids = set()

for phrase in search_phrases_intel_2:
    print(f"🔍 Processing search: {phrase}")

    for submission in subreddits_intel.search(query=phrase, sort="relevance", time_filter="all", limit=20):
        if submission.id in seen_ids or not submission.author:
            continue
        seen_ids.add(submission.id)

        author_name = submission.author.name
        replyGraph.add_node(author_name, subNum=1)
        dSubCommentId[submission.id] = author_name

        try:
            submission.comments.replace_more(limit=None)
            all_comments = submission.comments.list()

            # Sort comments by score, descending, and take top 300
            sorted_comments = sorted(
                [c for c in all_comments if c.author],
                key=lambda x: x.score,
                reverse=True
            )[:200]

            # First pass: index authors
            for comment in sorted_comments:
                dSubCommentId[comment.id] = comment.author.name

            # Second pass: add edges
            for comment in sorted_comments:
                parent_id = comment.parent_id.replace("t1_", "").replace("t3_", "")
                if parent_id in dSubCommentId:
                    src = comment.author.name
                    tgt = dSubCommentId[parent_id]

                    if src not in replyGraph:
                        replyGraph.add_node(src, subNum=0)
                    if tgt not in replyGraph:
                        replyGraph.add_node(tgt, subNum=0)

                    if replyGraph.has_edge(src, tgt):
                        replyGraph[src][tgt]['replyNum'] += 1
                    else:
                        replyGraph.add_edge(src, tgt, replyNum=1)

        except Exception as e:
            print(f"⚠️ Skipped submission {submission.id} due to error: {e}")

        time.sleep(3)  # Avoid rate limit

# --- Save graph ---
nx.write_graphml(replyGraph, "search_reply_graph_intel_2.graphml")
print(f"✅ Graph saved with {len(replyGraph.nodes)} nodes and {len(replyGraph.edges)} edges.")

🔍 Processing search: arc a750
🔍 Processing search: arc a750 review
🔍 Processing search: arc a750 launch
🔍 Processing search: arc a750 performance
🔍 Processing search: arc a750 benchmarks
🔍 Processing search: arc a750 gaming
🔍 Processing search: arc a750 issues
🔍 Processing search: arc a750 worth it
🔍 Processing search: arc a750 upgrade
🔍 Processing search: arc a750 vs
✅ Graph saved with 1029 nodes and 1921 edges.


## intel 3

In [171]:
gpu_keywords_intel = {
#     "arc a380": ("intel", "low")
#     "arc a750": ("intel", "mid"),
    "arc b580": ("intel", "high")
}



from itertools import product


contexts = ["", "review", "launch", "performance", "benchmarks", "gaming", "issues", "worth it", "upgrade", "vs"]

search_phrases_intel_3 = [f"{gpu} {ctx}".strip() for gpu, ctx in product(gpu_keywords_intel.keys(), contexts)]


In [172]:
del replyGraph
del dSubCommentId 

NameError: name 'replyGraph' is not defined

In [173]:
# intel 1 
replyGraph = nx.DiGraph()
dSubCommentId = {}
seen_ids = set()

for phrase in search_phrases_intel_2:
    print(f"🔍 Processing search: {phrase}")

    for submission in subreddits_intel.search(query=phrase, sort="relevance", time_filter="all", limit=20):
        if submission.id in seen_ids or not submission.author:
            continue
        seen_ids.add(submission.id)

        author_name = submission.author.name
        replyGraph.add_node(author_name, subNum=1)
        dSubCommentId[submission.id] = author_name

        try:
            submission.comments.replace_more(limit=None)
            all_comments = submission.comments.list()

            # Sort comments by score, descending, and take top 300
            sorted_comments = sorted(
                [c for c in all_comments if c.author],
                key=lambda x: x.score,
                reverse=True
            )[:200]

            # First pass: index authors
            for comment in sorted_comments:
                dSubCommentId[comment.id] = comment.author.name

            # Second pass: add edges
            for comment in sorted_comments:
                parent_id = comment.parent_id.replace("t1_", "").replace("t3_", "")
                if parent_id in dSubCommentId:
                    src = comment.author.name
                    tgt = dSubCommentId[parent_id]

                    if src not in replyGraph:
                        replyGraph.add_node(src, subNum=0)
                    if tgt not in replyGraph:
                        replyGraph.add_node(tgt, subNum=0)

                    if replyGraph.has_edge(src, tgt):
                        replyGraph[src][tgt]['replyNum'] += 1
                    else:
                        replyGraph.add_edge(src, tgt, replyNum=1)

        except Exception as e:
            print(f"⚠️ Skipped submission {submission.id} due to error: {e}")

        time.sleep(3)  # Avoid rate limit

# --- Save graph ---
nx.write_graphml(replyGraph, "search_reply_graph_intel_3.graphml")
print(f"✅ Graph saved with {len(replyGraph.nodes)} nodes and {len(replyGraph.edges)} edges.")

🔍 Processing search: arc b580
🔍 Processing search: arc b580 review
🔍 Processing search: arc b580 launch
🔍 Processing search: arc b580 performance
🔍 Processing search: arc b580 benchmarks
🔍 Processing search: arc b580 gaming
🔍 Processing search: arc b580 issues
🔍 Processing search: arc b580 worth it
🔍 Processing search: arc b580 upgrade
🔍 Processing search: arc b580 vs
✅ Graph saved with 2140 nodes and 4139 edges.


# Insights into the graphs 


### nvidia_tier_1 

In [2]:
import networkx as nx
import pandas as pd

def compute_centrality_from_graphml(graphml_path, output_name="graph"):
    """
    Load a .graphml file and compute centrality metrics.

    Parameters:
    - graphml_path: Path to the .graphml file
    - output_name: Filename prefix for CSV output

    Returns:
    - DataFrame of centrality metrics
    """
    # --- Load graph ---
    print(f"📂 Loading graph from {graphml_path}...")
    graph = nx.read_graphml(graphml_path)

    # --- Compute metrics ---
    degree = nx.degree_centrality(graph)
    betweenness = nx.betweenness_centrality(graph)
    eigenvector = nx.eigenvector_centrality(graph, max_iter=1000)
    closeness = nx.closeness_centrality(graph)

    # --- Combine into DataFrame ---
    df = pd.DataFrame({
        "User": list(degree.keys()),
        "Degree": list(degree.values()),
        "Betweenness": list(betweenness.values()),
        "Eigenvector": list(eigenvector.values()),
        "Closeness": list(closeness.values())
    })

    # --- Sort and save ---
    df = df.sort_values("Degree", ascending=False)
    out_csv = f"{output_name}_centrality_metrics.csv"
    df.to_csv(out_csv, index=False)
    print(f"✅ Exported metrics to {out_csv} with {len(df)} users.")

    return df


In [177]:
df = compute_centrality_from_graphml("search_reply_graph_4.graphml", output_name="search_reply_graph_6")
print(df.head(10))  # Show top 10 users




📂 Loading graph from search_reply_graph_4.graphml...
✅ Exported metrics to search_reply_graph_6_centrality_metrics.csv with 5521 users.
                    User    Degree  Betweenness  Eigenvector  Closeness
150          Nestledrink  0.270109     0.155000     0.755477   0.339070
167            KARMAAACS  0.027899     0.027595     0.065919   0.219100
1256       Stiven_Crysis  0.025000     0.000000     0.052341   0.214730
3123               m_w_h  0.022826     0.021181     0.102509   0.181626
0        PralineGold6868  0.020471     0.007616     0.000279   0.132300
789           nukleabomb  0.018659     0.007052     0.083937   0.216952
2357            Fraga500  0.014493     0.006091     0.011858   0.181844
368       rW0HgFyxoJhYka  0.013587     0.016981     0.037329   0.213066
5143  Certain_Active7773  0.013587     0.000875     0.013932   0.191666
4372         SadSpot8656  0.013587     0.012829     0.058302   0.207819


In [3]:
import networkx as nx

def calculate_centralities(graph_path):
    # Load the GraphML file
    G = nx.read_graphml(graph_path)

    # Ensure it's a directed graph if needed
    if not isinstance(G, nx.DiGraph):
        G = nx.DiGraph(G)

    # Calculate different centralities
    degree_centrality = nx.degree_centrality(G)
    in_degree_centrality = nx.in_degree_centrality(G)
    out_degree_centrality = nx.out_degree_centrality(G)
    closeness_centrality = nx.closeness_centrality(G)
    betweenness_centrality = nx.betweenness_centrality(G, normalized=True)
    eigenvector_centrality = nx.eigenvector_centrality(G, max_iter=1000)
    katz_centrality = nx.katz_centrality_numpy(G, alpha=0.1, beta=1.0)
    pagerank = nx.pagerank(G)

    # Combine results
    centralities = {}
    for node in G.nodes():
        centralities[node] = {
            'degree': degree_centrality.get(node, 0),
            'in_degree': in_degree_centrality.get(node, 0),
            'out_degree': out_degree_centrality.get(node, 0),
            'closeness': closeness_centrality.get(node, 0),
            'betweenness': betweenness_centrality.get(node, 0),
            'eigenvector': eigenvector_centrality.get(node, 0),
            'katz': katz_centrality.get(node, 0),
            'pagerank': pagerank.get(node, 0)
        }

    return centralities


In [180]:
results = calculate_centralities("search_reply_graph_4.graphml")

# Print top 5 nodes by PageRank
top_pagerank = sorted(results.items(), key=lambda x: x[1]['pagerank'], reverse=True)[:5]
for node, metrics in top_pagerank:
    print(f"User: {node}")
    for k, v in metrics.items():
        print(f"  {k}: {v:.4f}")


User: Nestledrink
  degree: 0.2701
  in_degree: 0.2652
  out_degree: 0.0049
  closeness: 0.3391
  betweenness: 0.1550
  eigenvector: 0.7555
  katz: 0.8107
  pagerank: 0.1096
User: nukleabomb
  degree: 0.0187
  in_degree: 0.0172
  out_degree: 0.0014
  closeness: 0.2170
  betweenness: 0.0071
  eigenvector: 0.0839
  katz: 0.0778
  pagerank: 0.0152
User: KARMAAACS
  degree: 0.0279
  in_degree: 0.0259
  out_degree: 0.0020
  closeness: 0.2191
  betweenness: 0.0276
  eigenvector: 0.0659
  katz: 0.0838
  pagerank: 0.0136
User: it_is_im
  degree: 0.0085
  in_degree: 0.0076
  out_degree: 0.0009
  closeness: 0.1946
  betweenness: 0.0053
  eigenvector: 0.0200
  katz: 0.0279
  pagerank: 0.0106
User: MajesticFunction8453
  degree: 0.0105
  in_degree: 0.0100
  out_degree: 0.0005
  closeness: 0.1740
  betweenness: 0.0005
  eigenvector: 0.0049
  katz: 0.0205
  pagerank: 0.0099


In [4]:
import networkx as nx
import numpy as np

def average_centrality(G, metric='degree'):
    """
    Compute the average centrality of a graph.

    Parameters:
        G (networkx.Graph): The graph to analyze.
        metric (str): The centrality metric to use. Options:
                      'degree', 'closeness', 'betweenness', 'eigenvector', 'katz'

    Returns:
        float: The average centrality value across all nodes.
    """

    if metric == 'degree':
        centrality = nx.degree_centrality(G)
    elif metric == 'closeness':
        centrality = nx.closeness_centrality(G)
    elif metric == 'betweenness':
        centrality = nx.betweenness_centrality(G)
    elif metric == 'eigenvector':
        centrality = nx.eigenvector_centrality(G, max_iter=1000)
    elif metric == 'katz':
        # Katz requires an alpha less than 1 / largest eigenvalue
        try:
            centrality = nx.katz_centrality(G, alpha=0.1, beta=1.0, max_iter=1000)
        except nx.NetworkXException as e:
            print(f"⚠️ Katz centrality error: {e}")
            return None
    else:
        raise ValueError("Unsupported metric. Choose from 'degree', 'closeness', 'betweenness', 'eigenvector', 'katz'.")

    return np.mean(list(centrality.values()))


In [183]:
G = nx.read_graphml("search_reply_graph_4.graphml")

metrics = ['degree', 'closeness', 'betweenness', 'eigenvector', 'katz']

for m in metrics:
    avg = average_centrality(G, metric=m)
    if avg is not None:
        print(f"{m.capitalize()} Centrality: {avg:.4f}")


Degree Centrality: 0.0006
Closeness Centrality: 0.0459
Betweenness Centrality: 0.0003
Eigenvector Centrality: 0.0019
Katz Centrality: 0.0041


## Nvidia_tier_2

In [187]:
df = compute_centrality_from_graphml("search_reply_graph_5.graphml", output_name="search_reply_graph_6")
print(df.head(5))  # Show top 10 users


📂 Loading graph from search_reply_graph_5.graphml...
✅ Exported metrics to search_reply_graph_6_centrality_metrics.csv with 4517 users.
                    User    Degree  Betweenness  Eigenvector  Closeness
0            Nestledrink  0.158547     0.144581     0.660567   0.308881
783   Arthur_Morgan44469  0.035651     0.033496     0.252679   0.218913
3182           bilal_hcg  0.023029     0.012721     0.032711   0.198023
646          kikimaru024  0.022365     0.021075     0.045208   0.215412
1587     Various_Ad_8448  0.021479     0.015342     0.063604   0.184011


In [186]:
del results 

In [188]:
results = calculate_centralities("search_reply_graph_5.graphml")

# Print top 5 nodes by PageRank
top_pagerank = sorted(results.items(), key=lambda x: x[1]['pagerank'], reverse=True)[:5]
for node, metrics in top_pagerank:
    print(f"User: {node}")
    for k, v in metrics.items():
        print(f"  {k}: {v:.4f}")


User: Nestledrink
  degree: 0.1585
  in_degree: 0.1499
  out_degree: 0.0086
  closeness: 0.3089
  betweenness: 0.1446
  eigenvector: 0.6606
  katz: 0.6949
  pagerank: 0.0693
User: JayomaW
  degree: 0.0016
  in_degree: 0.0011
  out_degree: 0.0004
  closeness: 0.2029
  betweenness: 0.0002
  eigenvector: 0.0194
  katz: 0.0208
  pagerank: 0.0307
User: Ajxtt
  degree: 0.0213
  in_degree: 0.0210
  out_degree: 0.0002
  closeness: 0.2606
  betweenness: 0.0004
  eigenvector: 0.1464
  katz: 0.1548
  pagerank: 0.0230
User: mtbhatch
  degree: 0.0004
  in_degree: 0.0002
  out_degree: 0.0002
  closeness: 0.1661
  betweenness: 0.0000
  eigenvector: 0.0025
  katz: 0.0058
  pagerank: 0.0130
User: Voodoo2-SLi
  degree: 0.0142
  in_degree: 0.0122
  out_degree: 0.0020
  closeness: 0.2391
  betweenness: 0.0201
  eigenvector: 0.1245
  katz: 0.1189
  pagerank: 0.0111


In [189]:
G = nx.read_graphml("search_reply_graph_5.graphml")

metrics = ['degree', 'closeness', 'betweenness', 'eigenvector', 'katz']

for m in metrics:
    avg = average_centrality(G, metric=m)
    if avg is not None:
        print(f"{m.capitalize()} Centrality: {avg:.4f}")


Degree Centrality: 0.0007
Closeness Centrality: 0.0492
Betweenness Centrality: 0.0004
Eigenvector Centrality: 0.0026
Katz Centrality: 0.0062


## Nvidia_tier_3 

In [190]:
df = compute_centrality_from_graphml("search_reply_graph_6.graphml", output_name="search_reply_graph_6")
print(df.head(5))  # Show top 10 users

df = compute_centrality_from_graphml("search_reply_graph_6.graphml", output_name="search_reply_graph_6")
print(df.head(5))  # Show top 10 users
​
📂 Loading graph from search_reply_graph_6.graphml...
✅ Exported metrics to search_reply_graph_6_centrality_metrics.csv with 4513 users.
                    User    Degree  Betweenness  Eigenvector  Closeness
0            Nestledrink  0.156472     0.143059     0.647564   0.309595
782   Arthur_Morgan44469  0.036126     0.033029     0.277132   0.219178
3177           bilal_hcg  0.023271     0.012692     0.033454   0.197298
636          kikimaru024  0.022385     0.020865     0.045187   0.215636
850          Bossman1086  0.021498     0.023815     0.131326   0.217248
del results 


📂 Loading graph from search_reply_graph_6.graphml...
✅ Exported metrics to search_reply_graph_6_centrality_metrics.csv with 4513 users.
                    User    Degree  Betweenness  Eigenvector  Closeness
0            Nestledrink  0.156472     0.143059     0.647564   0.309595
782   Arthur_Morgan44469  0.036126     0.033029     0.277132   0.219178
3177           bilal_hcg  0.023271     0.012692     0.033454   0.197298
636          kikimaru024  0.022385     0.020865     0.045187   0.215636
850          Bossman1086  0.021498     0.023815     0.131326   0.217248


In [191]:
del results 

In [192]:
results = calculate_centralities("search_reply_graph_6.graphml")

# Print top 5 nodes by PageRank
top_pagerank = sorted(results.items(), key=lambda x: x[1]['pagerank'], reverse=True)[:5]
for node, metrics in top_pagerank:
    print(f"User: {node}")
    for k, v in metrics.items():
        print(f"  {k}: {v:.4f}")


User: Nestledrink
  degree: 0.1565
  in_degree: 0.1478
  out_degree: 0.0086
  closeness: 0.3096
  betweenness: 0.1431
  eigenvector: 0.6476
  katz: 0.6886
  pagerank: 0.0677
User: Ajxtt
  degree: 0.0211
  in_degree: 0.0208
  out_degree: 0.0002
  closeness: 0.2606
  betweenness: 0.0002
  eigenvector: 0.1424
  katz: 0.1541
  pagerank: 0.0349
User: JayomaW
  degree: 0.0011
  in_degree: 0.0009
  out_degree: 0.0002
  closeness: 0.2029
  betweenness: 0.0000
  eigenvector: 0.0184
  katz: 0.0203
  pagerank: 0.0299
User: Voodoo2-SLi
  degree: 0.0142
  in_degree: 0.0122
  out_degree: 0.0020
  closeness: 0.2393
  betweenness: 0.0201
  eigenvector: 0.1212
  katz: 0.1185
  pagerank: 0.0110
User: bilal_hcg
  degree: 0.0233
  in_degree: 0.0206
  out_degree: 0.0027
  closeness: 0.1973
  betweenness: 0.0127
  eigenvector: 0.0335
  katz: 0.0635
  pagerank: 0.0104


In [193]:
G = nx.read_graphml("search_reply_graph_6.graphml")

metrics = ['degree', 'closeness', 'betweenness', 'eigenvector', 'katz']

for m in metrics:
    avg = average_centrality(G, metric=m)
    if avg is not None:
        print(f"{m.capitalize()} Centrality: {avg:.4f}")


Degree Centrality: 0.0007
Closeness Centrality: 0.0492
Betweenness Centrality: 0.0004
Eigenvector Centrality: 0.0026
Katz Centrality: 0.0063


## Amd_tier_1

In [194]:
df = compute_centrality_from_graphml("search_reply_graph_amd_1.graphml", output_name="search_reply_graph_amd_1")
print(df.head(5))  # Show top 10 users


📂 Loading graph from search_reply_graph_amd_1.graphml...
✅ Exported metrics to search_reply_graph_amd_1_centrality_metrics.csv with 1638 users.
               User    Degree  Betweenness   Eigenvector  Closeness
564   FlyingNipplez  0.089188     0.005818  2.284611e-03   0.060119
214          jecowa  0.065363     0.015269  4.921453e-05   0.039278
720        J_Echoes  0.063531     0.004665  9.572044e-06   0.057451
0            Uhmm69  0.051924     0.010488  9.550582e-09   0.050324
958  FantasticHydra  0.040929     0.000454  6.970269e-01   0.020300


In [195]:
del results 

In [196]:
results = calculate_centralities("search_reply_graph_amd_1.graphml")

# Print top 5 nodes by PageRank
top_pagerank = sorted(results.items(), key=lambda x: x[1]['pagerank'], reverse=True)[:5]
for node, metrics in top_pagerank:
    print(f"User: {node}")
    for k, v in metrics.items():
        print(f"  {k}: {v:.4f}")


User: FlyingNipplez
  degree: 0.0892
  in_degree: 0.0733
  out_degree: 0.0159
  closeness: 0.0601
  betweenness: 0.0058
  eigenvector: 0.0023
  katz: 0.3002
  pagerank: 0.0422
User: ZeroPaladn
  degree: 0.0305
  in_degree: 0.0287
  out_degree: 0.0018
  closeness: 0.0461
  betweenness: 0.0006
  eigenvector: 0.0000
  katz: 0.1063
  pagerank: 0.0293
User: jecowa
  degree: 0.0654
  in_degree: 0.0501
  out_degree: 0.0153
  closeness: 0.0393
  betweenness: 0.0153
  eigenvector: 0.0000
  katz: 0.2068
  pagerank: 0.0252
User: J_Echoes
  degree: 0.0635
  in_degree: 0.0556
  out_degree: 0.0079
  closeness: 0.0575
  betweenness: 0.0047
  eigenvector: 0.0000
  katz: 0.2030
  pagerank: 0.0251
User: HamsterOk3112
  degree: 0.0367
  in_degree: 0.0330
  out_degree: 0.0037
  closeness: 0.0685
  betweenness: 0.0049
  eigenvector: 0.0000
  katz: 0.1211
  pagerank: 0.0196


In [197]:
G = nx.read_graphml("search_reply_graph_amd_1.graphml")

metrics = ['degree', 'closeness', 'betweenness', 'eigenvector', 'katz']

for m in metrics:
    avg = average_centrality(G, metric=m)
    if avg is not None:
        print(f"{m.capitalize()} Centrality: {avg:.4f}")


Degree Centrality: 0.0017
Closeness Centrality: 0.0081
Betweenness Centrality: 0.0002
Eigenvector Centrality: 0.0030
Katz Centrality: 0.0210


## Amd_tier_2

In [202]:
df = compute_centrality_from_graphml("search_reply_graph_amd_2.graphml", output_name="search_reply_graph_amd_2")
print(df.head(5))  # Show top 10 users


📂 Loading graph from search_reply_graph_amd_2.graphml...
✅ Exported metrics to search_reply_graph_amd_2_centrality_metrics.csv with 1897 users.
                     User    Degree  Betweenness   Eigenvector  Closeness
969                jecowa  0.055907     0.040932  6.825584e-01   0.050438
196            apmspammer  0.053270     0.055266  2.222596e-04   0.076035
585  StardustNovaSynchron  0.045886     0.001790  1.719201e-07   0.062629
331        introvertpanda  0.032173     0.022924  4.359061e-04   0.065433
394             AtaliX_MC  0.031118     0.036257  4.284675e-04   0.071324


In [203]:
del results 

NameError: name 'results' is not defined

In [204]:
results = calculate_centralities("search_reply_graph_amd_2.graphml")

# Print top 5 nodes by PageRank
top_pagerank = sorted(results.items(), key=lambda x: x[1]['pagerank'], reverse=True)[:5]
for node, metrics in top_pagerank:
    print(f"User: {node}")
    for k, v in metrics.items():
        print(f"  {k}: {v:.4f}")


User: StardustNovaSynchron
  degree: 0.0459
  in_degree: 0.0432
  out_degree: 0.0026
  closeness: 0.0626
  betweenness: 0.0018
  eigenvector: 0.0000
  katz: 0.1734
  pagerank: 0.0327
User: apmspammer
  degree: 0.0533
  in_degree: 0.0522
  out_degree: 0.0011
  closeness: 0.0760
  betweenness: 0.0553
  eigenvector: 0.0002
  katz: 0.2096
  pagerank: 0.0227
User: jecowa
  degree: 0.0559
  in_degree: 0.0427
  out_degree: 0.0132
  closeness: 0.0504
  betweenness: 0.0409
  eigenvector: 0.6826
  katz: 0.2043
  pagerank: 0.0213
User: StandingEggs
  degree: 0.0264
  in_degree: 0.0232
  out_degree: 0.0032
  closeness: 0.0644
  betweenness: 0.0017
  eigenvector: 0.0002
  katz: 0.1091
  pagerank: 0.0162
User: AtaliX_MC
  degree: 0.0311
  in_degree: 0.0269
  out_degree: 0.0042
  closeness: 0.0713
  betweenness: 0.0363
  eigenvector: 0.0004
  katz: 0.1209
  pagerank: 0.0154


In [205]:
G = nx.read_graphml("search_reply_graph_amd_2.graphml")

metrics = ['degree', 'closeness', 'betweenness', 'eigenvector', 'katz']

for m in metrics:
    avg = average_centrality(G, metric=m)
    if avg is not None:
        print(f"{m.capitalize()} Centrality: {avg:.4f}")


Degree Centrality: 0.0015
Closeness Centrality: 0.0110
Betweenness Centrality: 0.0009
Eigenvector Centrality: 0.0025
Katz Centrality: 0.0203


## Amd_tier_3

In [208]:
df = compute_centrality_from_graphml("search_reply_graph_amd_3.graphml", output_name="search_reply_graph_amd_2")
print(df.head(5))  # Show top 10 users


📂 Loading graph from search_reply_graph_amd_3.graphml...
✅ Exported metrics to search_reply_graph_amd_2_centrality_metrics.csv with 4218 users.
                      User    Degree  Betweenness  Eigenvector  Closeness
135                Buksa07  0.036993     0.027759     0.671213   0.144906
2481     Few_Landscape1035  0.029405     0.028266     0.025561   0.195001
1203    Interceptor__Prime  0.028219     0.020374     0.026685   0.150023
1078  MysteriousSilentVoid  0.025848     0.029917     0.027838   0.173160
2211              Forazehy  0.025373     0.011678     0.016186   0.170252


In [209]:
del results 

In [210]:
results = calculate_centralities("search_reply_graph_amd_3.graphml")

# Print top 5 nodes by PageRank
top_pagerank = sorted(results.items(), key=lambda x: x[1]['pagerank'], reverse=True)[:5]
for node, metrics in top_pagerank:
    print(f"User: {node}")
    for k, v in metrics.items():
        print(f"  {k}: {v:.4f}")


User: FastRemigiusz
  degree: 0.0223
  in_degree: 0.0221
  out_degree: 0.0002
  closeness: 0.1493
  betweenness: 0.0002
  eigenvector: 0.0148
  katz: 0.0978
  pagerank: 0.0133
User: MaikyMoto
  degree: 0.0005
  in_degree: 0.0002
  out_degree: 0.0002
  closeness: 0.1282
  betweenness: 0.0000
  eigenvector: 0.0020
  katz: 0.0182
  pagerank: 0.0110
User: Forazehy
  degree: 0.0254
  in_degree: 0.0247
  out_degree: 0.0007
  closeness: 0.1703
  betweenness: 0.0117
  eigenvector: 0.0162
  katz: 0.1299
  pagerank: 0.0107
User: J_Echoes
  degree: 0.0247
  in_degree: 0.0218
  out_degree: 0.0028
  closeness: 0.1751
  betweenness: 0.0339
  eigenvector: 0.0137
  katz: 0.1324
  pagerank: 0.0102
User: HamsterOk3112
  degree: 0.0192
  in_degree: 0.0175
  out_degree: 0.0017
  closeness: 0.1650
  betweenness: 0.0167
  eigenvector: 0.0045
  katz: 0.1017
  pagerank: 0.0099


In [211]:
G = nx.read_graphml("search_reply_graph_amd_3.graphml")

metrics = ['degree', 'closeness', 'betweenness', 'eigenvector', 'katz']

for m in metrics:
    avg = average_centrality(G, metric=m)
    if avg is not None:
        print(f"{m.capitalize()} Centrality: {avg:.4f}")


Degree Centrality: 0.0008
Closeness Centrality: 0.0408
Betweenness Centrality: 0.0005
Eigenvector Centrality: 0.0020
Katz Centrality: 0.0117


## intel_tier2

In [216]:
df = compute_centrality_from_graphml("search_reply_graph_intel_2.graphml", output_name="search_reply_graph_intel_2")
print(df.head(5))  # Show top 10 users


📂 Loading graph from search_reply_graph_intel_2.graphml...
✅ Exported metrics to search_reply_graph_intel_2_centrality_metrics.csv with 1029 users.
                   User    Degree  Betweenness  Eigenvector  Closeness
233              jecowa  0.102140     0.003109     0.000030   0.078615
63      IntelArcTesting  0.077821     0.058488     0.500002   0.161370
496           6im6erbmw  0.052529     0.039383     0.066548   0.157772
77   Distinct-Race-2471  0.042802     0.041175     0.166093   0.148058
42           alvarkresh  0.041829     0.056261     0.162944   0.159689


In [217]:
del results 

In [218]:
results = calculate_centralities("search_reply_graph_intel_2.graphml")

# Print top 5 nodes by PageRank
top_pagerank = sorted(results.items(), key=lambda x: x[1]['pagerank'], reverse=True)[:5]
for node, metrics in top_pagerank:
    print(f"User: {node}")
    for k, v in metrics.items():
        print(f"  {k}: {v:.4f}")


User: jecowa
  degree: 0.1021
  in_degree: 0.0788
  out_degree: 0.0233
  closeness: 0.0786
  betweenness: 0.0031
  eigenvector: 0.0000
  katz: 0.2157
  pagerank: 0.0442
User: 6im6erbmw
  degree: 0.0525
  in_degree: 0.0457
  out_degree: 0.0068
  closeness: 0.1578
  betweenness: 0.0394
  eigenvector: 0.0665
  katz: 0.1412
  pagerank: 0.0182
User: Admirable-Raise-4942
  degree: 0.0029
  in_degree: 0.0019
  out_degree: 0.0010
  closeness: 0.1184
  betweenness: 0.0000
  eigenvector: 0.0133
  katz: 0.0287
  pagerank: 0.0173
User: akiionely
  degree: 0.0302
  in_degree: 0.0263
  out_degree: 0.0039
  closeness: 0.1132
  betweenness: 0.0018
  eigenvector: 0.0117
  katz: 0.0808
  pagerank: 0.0171
User: IntelArcTesting
  degree: 0.0778
  in_degree: 0.0477
  out_degree: 0.0302
  closeness: 0.1614
  betweenness: 0.0585
  eigenvector: 0.5000
  katz: 0.2428
  pagerank: 0.0164


In [219]:
G = nx.read_graphml("search_reply_graph_intel_2.graphml")

metrics = ['degree', 'closeness', 'betweenness', 'eigenvector', 'katz']

for m in metrics:
    avg = average_centrality(G, metric=m)
    if avg is not None:
        print(f"{m.capitalize()} Centrality: {avg:.4f}")


Degree Centrality: 0.0036
Closeness Centrality: 0.0506
Betweenness Centrality: 0.0011
Eigenvector Centrality: 0.0105
Katz Centrality: 0.0262


## intel_tier_1

In [5]:
## intel 

df = compute_centrality_from_graphml("search_reply_graph_intel_1.graphml", output_name="search_reply_graph_intel_1")
print(df.head(5))  # Show top 10 users




📂 Loading graph from search_reply_graph_intel_1.graphml...
✅ Exported metrics to search_reply_graph_intel_1_centrality_metrics.csv with 821 users.
              User    Degree  Betweenness  Eigenvector  Closeness
116         jecowa  0.132927     0.036404     0.017409   0.065803
624  AutoModerator  0.058537     0.002923     0.000149   0.082519
378      6im6erbmw  0.057317     0.030967     0.186287   0.108147
25      alvarkresh  0.048780     0.070633     0.437162   0.091952
0        JayIsLoco  0.040244     0.020733     0.327637   0.088305


In [7]:
# del results 

results = calculate_centralities("search_reply_graph_intel_1.graphml")

# Print top 5 nodes by PageRank
top_pagerank = sorted(results.items(), key=lambda x: x[1]['pagerank'], reverse=True)[:5]
for node, metrics in top_pagerank:
    print(f"User: {node}")
    for k, v in metrics.items():
        print(f"  {k}: {v:.4f}")

User: jecowa
  degree: 0.1329
  in_degree: 0.1012
  out_degree: 0.0317
  closeness: 0.0658
  betweenness: 0.0364
  eigenvector: 0.0174
  katz: 0.3119
  pagerank: 0.0518
User: jackrabbit1994
  degree: 0.0293
  in_degree: 0.0220
  out_degree: 0.0073
  closeness: 0.0957
  betweenness: 0.0070
  eigenvector: 0.0062
  katz: 0.0766
  pagerank: 0.0338
User: 6im6erbmw
  degree: 0.0573
  in_degree: 0.0476
  out_degree: 0.0098
  closeness: 0.1081
  betweenness: 0.0310
  eigenvector: 0.1863
  katz: 0.1660
  pagerank: 0.0304
User: Corentinrobin29
  degree: 0.0159
  in_degree: 0.0146
  out_degree: 0.0012
  closeness: 0.0412
  betweenness: 0.0000
  eigenvector: 0.0001
  katz: 0.0580
  pagerank: 0.0177
User: cclambert95
  degree: 0.0037
  in_degree: 0.0024
  out_degree: 0.0012
  closeness: 0.0926
  betweenness: 0.0026
  eigenvector: 0.0013
  katz: 0.0476
  pagerank: 0.0149


In [8]:
G = nx.read_graphml("search_reply_graph_intel_1.graphml")

metrics = ['degree', 'closeness', 'betweenness', 'eigenvector', 'katz']

for m in metrics:
    avg = average_centrality(G, metric=m)
    if avg is not None:
        print(f"{m.capitalize()} Centrality: {avg:.4f}")


Degree Centrality: 0.0038
Closeness Centrality: 0.0215
Betweenness Centrality: 0.0010
Eigenvector Centrality: 0.0096
Katz Centrality: 0.0312


## intel_tier_3


In [212]:
df = compute_centrality_from_graphml("search_reply_graph_intel_3.graphml", output_name="search_reply_graph_intel_3")
print(df.head(5))  # Show top 10 users


📂 Loading graph from search_reply_graph_intel_3.graphml...
✅ Exported metrics to search_reply_graph_intel_3_centrality_metrics.csv with 2140 users.
                    User    Degree  Betweenness  Eigenvector  Closeness
914            kylinblue  0.056568     0.000000     0.003549   0.204699
73   ExcitementGrand2663  0.053296     0.038682     0.707995   0.125138
275        MyHonestViews  0.048621     0.017060     0.025861   0.220335
650           apmspammer  0.047218     0.000393     0.001152   0.180540
584          DeathDexoys  0.036933     0.030068     0.007953   0.197114


In [213]:
del results 

In [214]:
results = calculate_centralities("search_reply_graph_intel_3.graphml")

# Print top 5 nodes by PageRank
top_pagerank = sorted(results.items(), key=lambda x: x[1]['pagerank'], reverse=True)[:5]
for node, metrics in top_pagerank:
    print(f"User: {node}")
    for k, v in metrics.items():
        print(f"  {k}: {v:.4f}")


User: apmspammer
  degree: 0.0472
  in_degree: 0.0463
  out_degree: 0.0009
  closeness: 0.1805
  betweenness: 0.0004
  eigenvector: 0.0012
  katz: 0.1446
  pagerank: 0.0400
User: Helpful-Option-3047
  degree: 0.0360
  in_degree: 0.0355
  out_degree: 0.0005
  closeness: 0.1997
  betweenness: 0.0004
  eigenvector: 0.1207
  katz: 0.1602
  pagerank: 0.0226
User: SmartHost7823
  degree: 0.0304
  in_degree: 0.0281
  out_degree: 0.0023
  closeness: 0.2048
  betweenness: 0.0323
  eigenvector: 0.0075
  katz: 0.1296
  pagerank: 0.0215
User: Ledot3
  degree: 0.0009
  in_degree: 0.0005
  out_degree: 0.0005
  closeness: 0.1605
  betweenness: 0.0000
  eigenvector: 0.0168
  katz: 0.0270
  pagerank: 0.0189
User: MyHonestViews
  degree: 0.0486
  in_degree: 0.0477
  out_degree: 0.0009
  closeness: 0.2203
  betweenness: 0.0171
  eigenvector: 0.0259
  katz: 0.2026
  pagerank: 0.0187


In [215]:
G = nx.read_graphml("search_reply_graph_intel_3.graphml")

metrics = ['degree', 'closeness', 'betweenness', 'eigenvector', 'katz']

for m in metrics:
    avg = average_centrality(G, metric=m)
    if avg is not None:
        print(f"{m.capitalize()} Centrality: {avg:.4f}")


Degree Centrality: 0.0018
Closeness Centrality: 0.0512
Betweenness Centrality: 0.0007
Eigenvector Centrality: 0.0031
Katz Centrality: 0.0163
